### Устанавливаем Implicit, смотрим на данные

In [1]:
#!pip3 install implicit

In [2]:
from os import path, environ
import pandas as pd
import numpy as np

In [3]:
data_dir = "/Users/dmitry/Desktop/Lab 2"
environ["DATA_DIR"] = "/Users/dmitry/Desktop/Lab 2"
data = data = pd.read_csv('lastfm_small.tsv', sep='\t', encoding='utf-8', header=None)
data.columns = ["user", "artist-mbid", "artist-name", "total-plays"]
data.head()

,user,artist-mbid,artist-name,total-plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706


In [4]:
data.fillna("None", inplace=True)
data["user_id"] = data["user"].astype("category").cat.codes.copy() + 1
data["artist_id"] = data["artist-mbid"].astype("category").cat.codes.copy() + 1
data.drop(["artist-name", "artist-mbid", "user"], axis=1, inplace=True)
data.head()

,total-plays,user_id,artist_id
0,2137,1,15531
1,1099,1,63469
2,897,1,46858
3,717,1,15968
4,706,1,48969


In [5]:
data.describe()

,total-plays,user_id,artist_id
count,1000000.000000,1000000.000000,1000000.000000
mean,216.606950,10232.925996,33678.492236
std,604.378024,5912.022447,19230.330182
min,1.000000,1.000000,1.000000
25%,34.000000,5118.000000,17298.000000
50%,94.000000,10237.000000,34544.000000
75%,225.000000,15347.000000,49488.000000
max,135392.000000,20465.000000,66799.000000


### Разбиваем датасет на обучающую и тестовую выборки

In [6]:
test_indices = np.random.choice(
    data.index.values,
    replace=False,
    size=int(len(data.index.values) * 0.2)
)
test_data = data.iloc[test_indices]
train_data = data.drop(test_indices)

In [7]:
test_user_set = set(test_data["user_id"].unique())
train_user_set = set(train_data["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(
    len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(
    len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 1
нет в тестовой выборке, но есть в обучающей: 26
всего пользователей: 20465


In [8]:
user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
bad_indices = test_data[test_data["user_id"].isin(user_ids_to_exclude).values].index
test_data.drop(bad_indices, inplace=True)
bad_indices = train_data[train_data["user_id"].isin(user_ids_to_exclude).values]
train_data.drop(bad_indices.index, inplace=True)

/Users/dmitry/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
test_file_name = "lastfm.test.0"
test_data[["user_id", "artist_id", "total-plays"]].to_csv(
    path.join(data_dir, test_file_name),
    sep="\t",
    header=False,
    index=False
)
train_file_name = "train/lastfm.train.0"
train_data[["user_id", "artist_id", "total-plays"]].to_csv(
    path.join(data_dir, train_file_name),
    sep="\t",
    header=False,
    index=False
)

### Создаём разреженную матрицу item*user

In [10]:
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np

plays = coo_matrix((
    train_data["total-plays"].astype(np.float32),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

# функция, которая красиво печатает информацию о разреженных матрицах
def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))
sparse_info(plays.tocsr())

Размерности матрицы: (66800, 20466)
Ненулевых элементов в матрице: 799784
Доля ненулевых элементов: 0.0005850099858916
Среднее значение ненулевых элементов: 216.72438049316406
Максимальное значение ненулевых элементов: 135392.0
Минимальное значение ненулевых элементов: 1.0


### kNN

In [11]:
import time
from implicit.nearest_neighbours import CosineRecommender

In [12]:
model = CosineRecommender()
model.fit(plays)
sparse_info(model.similarity)

Размерности матрицы: (66800, 66800)
Ненулевых элементов в матрице: 1215581
Доля ненулевых элементов: 0.00027241497543834485
Среднее значение ненулевых элементов: 0.44413862566940154
Максимальное значение ненулевых элементов: 1.0000000582571396
Минимальное значение ненулевых элементов: 0.0


In [13]:
start = time.time()
user_plays = plays.T.tocsr()
with open(path.join(data_dir, "recs_knn", test_file_name + ".recs.tsv"), "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секнуд".format(
        time.time() - start))

получили рекомендации для всех пользователей за 29.099152088165283 секнуд


### ALS

In [14]:
from implicit.als import AlternatingLeastSquares

In [15]:
plays = coo_matrix((
    train_data["total-plays"].astype('double'),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

model = AlternatingLeastSquares(factors=50)
model.fit(plays)

In [16]:
start = time.time()
user_plays = plays.T.tocsr()
with open(path.join(data_dir, "recs_als", test_file_name + ".recs.tsv"), "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секнуд".format(
        time.time() - start))

получили рекомендации для всех пользователей за 78.05753779411316 секнуд


### MREC evaluating

kNN:

In [17]:
!mrec_evaluate --input_format=tsv --test_input_format=tsv --train lastfm.test.0 --recsdir recs_knn

[2017-12-10 23:22:56,576] INFO: processing /Users/dmitry/Desktop/Lab 2/lastfm.test.0...
None
mrr            0.0486 +/- 0.0000
prec@5         0.0158 +/- 0.0000
prec@10        0.0145 +/- 0.0000
prec@15        0.0097 +/- 0.0000
prec@20        0.0073 +/- 0.0000


ALS:

In [18]:
!mrec_evaluate --input_format=tsv --test_input_format=tsv --train lastfm.test.0 --recsdir recs_als

[2017-12-10 23:23:05,187] INFO: processing /Users/dmitry/Desktop/Lab 2/lastfm.test.0...
None
mrr            0.2959 +/- 0.0000
prec@5         0.1293 +/- 0.0000
prec@10        0.1071 +/- 0.0000
prec@15        0.0714 +/- 0.0000
prec@20        0.0535 +/- 0.0000
